# Building a Natively Multimodal RAG Pipeline (over a Slide Deck)

<a href="https://colab.research.google.com/github/run-llama/llama_parse/blob/main/examples/multimodal/multimodal_rag_slide_deck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this cookbook we show you how to build a multimodal RAG pipeline over a slide deck, with text, tables, images, diagrams, and complex layouts.

A gap of text-based RAG is that they struggle with purely text-based representations of complex documents. For instance, if a page contains a lot of images and diagrams, a text parser would need to rely on raw OCR to extract out text. You can also use a multimodal model (e.g. gpt-4o and up) to do text extraction, but this is inherently a lossy conversion.

Instead a **native multimodal pipeline** stores both a text and image representation of a document chunk. They are indexed via embeddings (text or image), and during synthesis both text and image are directly fed to the multimodal model for synthesis.

This can have the following advantages:
- **Robustness**: This solution is more robust than a pure text or even a pure image-based approach. In a pure text RAG approach, the parsing piece can be lossy. In a pure image-based approach, multimodal OCR is not perfect and may lose out against text parsing for text-heavy documents.
- **Cost Optimization**: You may choose to dynamically include text-only, or text + image depending on the content of the page.

![mm_rag_diagram](./multimodal_rag_slide_deck_img.png)

## Setup

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Setup Observability

We setup an integration with LlamaTrace (integration with Arize).

If you haven't already done so, make sure to create an account here: https://llamatrace.com/login. Then create an API key and put it in the `PHOENIX_API_KEY` variable below.

In [ ]:
!pip install -U llama-index-callbacks-arize-phoenix

In [ ]:
# setup Arize Phoenix for logging/observability
import llama_index.core
import os

PHOENIX_API_KEY = "<PHOENIX_API_KEY>"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)

### Load Data

Here we load the [Conoco Phillips 2023 investor meeting slide deck](https://static.conocophillips.com/files/2023-conocophillips-aim-presentation.pdf).

In [ ]:
!mkdir data
!mkdir data_images
!wget "https://static.conocophillips.com/files/2023-conocophillips-aim-presentation.pdf" -O data/conocophillips.pdf

### Model Setup

Setup models that will be used for downstream orchestration.

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-large")
llm = OpenAI(model="gpt-4o")

Settings.embed_model = embed_model
Settings.llm = llm

## Use LlamaParse to Parse Text and Images

In this example, use LlamaParse to parse both the text and images from the document.

We parse out the text in two ways: 
- in regular `text` mode using our default text layout algorithm
- in `markdown` mode using GPT-4o (`gpt4o_mode=True`). This also allows us to capture page screenshots

In [ ]:
from llama_parse import LlamaParse


parser_text = LlamaParse(result_type="text")
parser_gpt4o = LlamaParse(result_type="markdown", gpt4o_mode=True)

In [ ]:
print(f"Parsing text...")
docs_text = parser_text.load_data("data/conocophillips.pdf")
print(f"Parsing PDF file...")
md_json_objs = parser_gpt4o.get_json_result("data/conocophillips.pdf")
md_json_list = md_json_objs[0]["pages"]

Parsing text...
Started parsing the file under job_id 62f157a9-9ef9-4e5b-95ac-67093fa25800
..........Parsing PDF file...
Started parsing the file under job_id 1ddd5654-062b-4e19-b488-d66efc9c509d


In [ ]:
print(md_json_list[10]["md"])

# Commitment to Disciplined Reinvestment Rate

| Period       | Description                          | Reinvestment Rate | WTI Average |
|--------------|--------------------------------------|-------------------|-------------|
| 2012-2016    | Industry Growth Focus                | >100%             | ~$75/BBL    |
| 2017-2022    | ConocoPhillips Strategy Reset        | <60%              | ~$63/BBL    |
| 2023E        |                                      |                   | at $80/BBL  |
| 2024-2028    | Disciplined Reinvestment Rate        | ~50%              | at $60/BBL  |
| 2029-2032    |                                      | ~6% CFO CAGR      | at $60/BBL  |

- **Historic Reinvestment Rate**: Gray bars
- **Reinvestment Rate at $60/BBL WTI**: Blue bars
- **Reinvestment Rate at $80/BBL WTI**: Dashed blue lines

Reinvestment rate and cash from operations (CFO) are non-GAAP measures. Definitions and reconciliations are included in the Appendix.


In [ ]:
image_dicts = parser_gpt4o.get_images(md_json_objs, download_path="data_images")

## Build Multimodal Index

In this section we build the multimodal index over the parsed deck. 

We do this by creating **text** nodes from the document that contain metadata referencing the original image path.

In this example we're indexing the text node for retrieval. The text node has a reference to both the parsed text as well as the image screenshot.

#### Get Text Nodes

In [ ]:
from llama_index.core.schema import TextNode
from typing import Optional

In [ ]:
# get pages loaded through llamaparse
import re


def get_page_number(file_name):
    match = re.search(r"-page-(\d+)\.jpg$", str(file_name))
    if match:
        return int(match.group(1))
    return 0


def _get_sorted_image_files(image_dir):
    """Get image files sorted by page."""
    raw_files = [f for f in list(Path(image_dir).iterdir()) if f.is_file()]
    sorted_files = sorted(raw_files, key=get_page_number)
    return sorted_files

In [ ]:
from copy import deepcopy
from pathlib import Path


# attach image metadata to the text nodes
def get_text_nodes(docs, image_dir=None, json_dicts=None):
    """Split docs into nodes, by separator."""
    nodes = []

    image_files = _get_sorted_image_files(image_dir) if image_dir is not None else None
    md_texts = [d["md"] for d in json_dicts] if json_dicts is not None else None

    doc_chunks = [c for d in docs for c in d.text.split("---")]
    for idx, doc_chunk in enumerate(doc_chunks):
        chunk_metadata = {"page_num": idx + 1}
        if image_files is not None:
            image_file = image_files[idx]
            chunk_metadata["image_path"] = str(image_file)
        if md_texts is not None:
            chunk_metadata["parsed_text_markdown"] = md_texts[idx]
        chunk_metadata["parsed_text"] = doc_chunk
        node = TextNode(
            text="",
            metadata=chunk_metadata,
        )
        nodes.append(node)

    return nodes

In [ ]:
# this will split into pages
text_nodes = get_text_nodes(docs_text, image_dir="data_images", json_dicts=md_json_list)

In [ ]:
print(text_nodes[10].get_content(metadata_mode="all"))

page_num: 11
image_path: data_images/1ddd5654-062b-4e19-b488-d66efc9c509d-page_39.jpg
parsed_text_markdown: # Commitment to Disciplined Reinvestment Rate

| Period       | Description                          | Reinvestment Rate | WTI Average |
|--------------|--------------------------------------|-------------------|-------------|
| 2012-2016    | Industry Growth Focus                | >100%             | ~$75/BBL    |
| 2017-2022    | ConocoPhillips Strategy Reset        | <60%              | ~$63/BBL    |
| 2023E        |                                      |                   | at $80/BBL  |
| 2024-2028    | Disciplined Reinvestment Rate        | ~50%              | at $60/BBL  |
| 2029-2032    |                                      | ~6% CFO CAGR      | at $60/BBL  |

- **Historic Reinvestment Rate**: Gray bars
- **Reinvestment Rate at $60/BBL WTI**: Blue bars
- **Reinvestment Rate at $80/BBL WTI**: Dashed blue lines

Reinvestment rate and cash from operations (CFO) are non-GAAP

#### Build Index

Once the text nodes are ready, we feed into our vector store index abstraction, which will index these nodes into a simple in-memory vector store (of course, you should definitely check out our 40+ vector store integrations!)

In [ ]:
import os
from llama_index.core import (
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)

if not os.path.exists("storage_nodes"):
    index = VectorStoreIndex(text_nodes, embed_model=embed_model)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage_nodes")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage_nodes")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

retriever = index.as_retriever()

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/tiktoken/core.py:50: RuntimeWarning: coroutine 'LlamaParse.aload_data' was never awaited
  self._core_bpe = _tiktoken.CoreBPE(mergeable_ranks, special_tokens, pat_str)


## Build Multimodal Query Engine

We now use LlamaIndex abstractions to build a **custom query engine**. In contrast to a standard RAG query engine that will retrieve the text node and only put that into the prompt (response synthesis module), this custom query engine will also load the image document, and put both the text and image document into the response synthesis module.

In [ ]:
from llama_index.core.query_engine import CustomQueryEngine, SimpleMultiModalQueryEngine
from llama_index.core.retrievers import BaseRetriever
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.core.schema import ImageNode, NodeWithScore, MetadataMode
from llama_index.core.prompts import PromptTemplate
from llama_index.core.base.response.schema import Response
from typing import Optional


gpt_4o = OpenAIMultiModal(model="gpt-4o", max_new_tokens=4096)

QA_PROMPT_TMPL = """\
Below we give parsed text from slides in two different formats, as well as the image.

We parse the text in both 'markdown' mode as well as 'raw text' mode. Markdown mode attempts \
to convert relevant diagrams into tables, whereas raw text tries to maintain the rough spatial \
layout of the text.

Use the image information first and foremost. ONLY use the text/markdown information 
if you can't understand the image.

---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query. Explain whether you got the answer
from the parsed markdown or raw text or image, and if there's discrepancies, and your reasoning for the final answer.

Query: {query_str}
Answer: """

QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)


class MultimodalQueryEngine(CustomQueryEngine):
    """Custom multimodal Query Engine.

    Takes in a retriever to retrieve a set of document nodes.
    Also takes in a prompt template and multimodal model.

    """

    qa_prompt: PromptTemplate
    retriever: BaseRetriever
    multi_modal_llm: OpenAIMultiModal

    def __init__(self, qa_prompt: Optional[PromptTemplate] = None, **kwargs) -> None:
        """Initialize."""
        super().__init__(qa_prompt=qa_prompt or QA_PROMPT, **kwargs)

    def custom_query(self, query_str: str):
        # retrieve text nodes
        nodes = self.retriever.retrieve(query_str)
        # create ImageNode items from text nodes
        image_nodes = [
            NodeWithScore(node=ImageNode(image_path=n.metadata["image_path"]))
            for n in nodes
        ]

        # create context string from text nodes, dump into the prompt
        context_str = "\n\n".join(
            [r.get_content(metadata_mode=MetadataMode.LLM) for r in nodes]
        )
        fmt_prompt = self.qa_prompt.format(context_str=context_str, query_str=query_str)

        # synthesize an answer from formatted text and images
        llm_response = self.multi_modal_llm.complete(
            prompt=fmt_prompt,
            image_documents=[image_node.node for image_node in image_nodes],
        )
        return Response(
            response=str(llm_response),
            source_nodes=nodes,
            metadata={"text_nodes": text_nodes, "image_nodes": image_nodes},
        )

        return response

In [ ]:
query_engine = MultimodalQueryEngine(
    retriever=index.as_retriever(similarity_top_k=9), multi_modal_llm=gpt_4o
)

### Define Baseline

In addition, we define a "baseline" where we rely only on text-based indexing. Here we define an index using only the nodes that are parsed in text-mode from LlamaParse. 

**NOTE**: We don't currently include the markdown-parsed text because that was parsed with GPT-4o, so already uses a multimodal model during the text extraction phase.

It is of course a valid experiment to compare RAG where multimodal extraction only happens during indexing, vs. the current multimodal RAG implementation where images are fed during synthesis to the LLM. 

In [ ]:
def get_nodes(docs):
    """Split docs into nodes, by separator."""
    nodes = []
    for doc in docs:
        doc_chunks = doc.text.split("\n---\n")
        for doc_chunk in doc_chunks:
            node = TextNode(
                text=doc_chunk,
                metadata=deepcopy(doc.metadata),
            )
            nodes.append(node)

    return nodes

In [ ]:
base_nodes = get_nodes(docs_text)

In [ ]:
print(base_nodes[13].get_content(metadata_mode="all"))

Our Differentiated Portfolio: Deep; Durable and Diverse
                              20 BBOE of Resource                                           Diverse Production Base
                            Under $40/BBL Cost of Supply                              10-Year Plan Cumulative Production (BBOE)
      S50                   S32/BBL                                                Lower 48                           Alaska
                    Average Cost of Supply
  3 $40                                                                                                                       GKA        GWA
                                                                                                                      GPA     WNS
      $30                                                                                                             EMENA
  3                                                                                                                              Norway
 

In [ ]:
base_index = VectorStoreIndex(base_nodes, embed_model=embed_model)
base_query_engine = base_index.as_query_engine(llm=llm, similarity_top_k=9)

## Build a Multimodal Agent

Build an agent around the multimodal query engine. This gives you agent capabilities like query planning/decomposition and memory around a central QA interface.

In [ ]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.agent import FunctionCallingAgentWorker


vector_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="vector_tool",
    description=(
        "Useful for retrieving specific context from the data. Do NOT select if question asks for a summary of the data."
    ),
)
agent = FunctionCallingAgentWorker.from_tools(
    [vector_tool], llm=llm, verbose=True
).as_agent()

In [ ]:
# define a similar agent for the baseline
base_vector_tool = QueryEngineTool.from_defaults(
    query_engine=base_query_engine,
    name="vector_tool",
    description=(
        "Useful for retrieving specific context from the data. Do NOT select if question asks for a summary of the data."
    ),
)
base_agent = FunctionCallingAgentWorker.from_tools(
    [base_vector_tool], llm=llm, verbose=True
).as_agent()

## Try out Queries

Let's try out queries against these documents and compare against each other.

In [ ]:
query = (
    "Tell me about the diverse geographies where Conoco Phillips has a production base"
)
response = agent.query(query)
base_response = base_agent.query(query)

Added user message to memory: Tell me about the diverse geographies where Conoco Phillips has a production base
=== Calling Function ===
Calling function: vector_tool with args: {"input": "Conoco Phillips production base geographies"}
=== Function Output ===
ConocoPhillips' production base geographies include:

1. **Lower 48** (Permian, Eagle Ford, Bakken, Other)
2. **Alaska** (GKA, GWA, GPA, WNS)
3. **EMENA** (Norway, Libya, Qatar)
4. **Asia Pacific** (APLNG, Malaysia, China)
5. **Canada** (Montney, Surmont)

This information was derived from the image on page 14, which provides a detailed breakdown of the diverse production base and the regions involved. The parsed markdown and raw text also support this information, but the image provides the clearest and most comprehensive view. There are no discrepancies between the image and the parsed text in this case.
=== LLM Response ===
ConocoPhillips has a diverse production base spread across various geographies, including:

1. **Lower 48*

In [ ]:
print(str(response))

ConocoPhillips has a diverse production base spread across various geographies, including:

1. **Lower 48**:
   - Permian Basin
   - Eagle Ford
   - Bakken
   - Other regions within the continental United States

2. **Alaska**:
   - Greater Kuparuk Area (GKA)
   - Greater Prudhoe Area (GPA)
   - Greater Willow Area (GWA)
   - Western North Slope (WNS)

3. **EMENA (Europe, Middle East, and North Africa)**:
   - Norway
   - Libya
   - Qatar

4. **Asia Pacific**:
   - Australia Pacific LNG (APLNG)
   - Malaysia
   - China

5. **Canada**:
   - Montney
   - Surmont

These regions highlight the global reach and diverse geographical footprint of ConocoPhillips' production operations.


In [ ]:
print(response.source_nodes[7].get_content(metadata_mode="all"))

page_num: 14
image_path: data_images/1ddd5654-062b-4e19-b488-d66efc9c509d-page_12.jpg
parsed_text_markdown: # Our Differentiated Portfolio: Deep, Durable and Diverse

## ~20 BBOE of Resource
Under $40/BBL Cost of Supply

### ~ $32/BBL
Average Cost of Supply

### WTI Cost of Supply ($/BBL)

| Cost ($/BBL) | Resource (BBOE) |
|--------------|-----------------|
| $0           | 0               |
| $10          |                 |
| $20          |                 |
| $30          |                 |
| $40          |                 |
| $50          |                 |

- **Legend:**
  - Lower 48
  - Canada
  - Alaska
  - EMENA
  - Asia Pacific

*Costs assume a mid-cycle price environment of $60/BBL WTI.*

## Diverse Production Base
10-Year Plan Cumulative Production (BBOE)

| Region       | Sub-region      |
|--------------|-----------------|
| Lower 48     | Permian         |
|              | Eagle Ford      |
|              | Bakken          |
|              | Other           |
| Alaska 

In [ ]:
print(str(base_response))

ConocoPhillips has a diverse production base spanning several key geographies:

1. **Lower 48 (United States)**: This includes major production areas such as the Permian Basin, Bakken Formation, and Eagle Ford Shale.
2. **Alaska**: Significant operations in the North Slope region.
3. **Canada**: Operations in the Montney Formation and the Surmont oil sands project.
4. **EMENA (Europe, Middle East, and North Africa)**: Notable operations in Norway and Libya.
5. **Asia Pacific**: Includes operations in Malaysia, China, and the Australia Pacific LNG (APLNG) project.
6. **Qatar**: Involvement in the country's energy sector.

These regions highlight the company's extensive and varied geographical footprint in the energy production industry.


In [ ]:
print(base_response.source_nodes[1].get_content(metadata_mode="all"))

Our Differentiated Portfolio: Deep; Durable and Diverse
                              20 BBOE of Resource                                           Diverse Production Base
                            Under $40/BBL Cost of Supply                              10-Year Plan Cumulative Production (BBOE)
      S50                   S32/BBL                                                Lower 48                           Alaska
                    Average Cost of Supply
  3 $40                                                                                                                       GKA        GWA
                                                                                                                      GPA     WNS
      $30                                                                                                             EMENA
  3                                                                                                                              Norway
 